# 46 Clothing Categories

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
pip install git+https://github.com/fastai/fastai.git

  Cloning https://github.com/fastai/fastai.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-his9304w
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/fastai/fastai.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-his9304w'
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\User\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-req-build-his9304w\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-req-build-his9304w\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-pip-egg-info-mdp_kn_s'
         cwd: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-his9304w\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\Public\Documents\ESTsoft\CreatorTe

We import all the necessary packages. We are going to work with the [fastai V1 library](http://www.fast.ai/2018/10/02/fastai-ai/) which sits on top of [Pytorch 1.0](https://hackernoon.com/pytorch-1-0-468332ba5163). The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [3]:
from fastai import *
from fastai.vision.all import *

In [4]:
# torch.cuda.set_device(0)

## Looking at the data

Classification of Cloth Categories

Large-scale Fashion Recognition and Retrieval (DeepFashion) Dataset 
Category and Attribute Prediction Benchmark

By Multimedia Lab, The Chinese University of Hong Kong


For more information about the dataset, visit the project website:

  http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html


  @inproceedings{liu2016deepfashion,
 	author = {Ziwei Liu, Ping Luo, Shi Qiu, Xiaogang Wang, and Xiaoou Tang},
 	title = {DeepFashion: Powering Robust Clothes Recognition and Retrieval with Rich Annotations},
 	booktitle = {Proceedings of IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
 	month = June,
 	year = {2016} 
  }

In [5]:
path = Path("data/cloth_categories/")

In [6]:
img_dir_path = path/"img"
len(img_dir_path.ls())

5621

In [7]:
fnames = get_image_files(img_dir_path/"Zip-Up_Puffer_Jacket")
fnames[:5]

(#5) [Path('data/cloth_categories/img/img/Zip-Up_Puffer_Jacket/img_00000001.jpg'),Path('data/cloth_categories/img/img/Zip-Up_Puffer_Jacket/img_00000002.jpg'),Path('data/cloth_categories/img/img/Zip-Up_Puffer_Jacket/img_00000003.jpg'),Path('data/cloth_categories/img/img/Zip-Up_Puffer_Jacket/img_00000004.jpg'),Path('data/cloth_categories/img/img/Zip-Up_Puffer_Jacket/img_00000005.jpg')]

In [17]:
from fastai import *
from fastai.vision.all import *

In [19]:
np.random.seed(42)
data = ImageDataBunch.from_csv(path, csv_labels="train_labels.csv", ds_tfms=get_transforms(), size=150)
data.normalize(imagenet_stats)

NameError: name 'ImageDataBunch' is not defined

In [ ]:
data.show_batch(rows=6, figsize=(14,12))

In [ ]:
print(data.classes)
len(data.classes),data.c

## Training: resnet34

Now we will start training our model. We will use a [convolutional neural network](http://cs231n.github.io/convolutional-networks/) backbone and a fully connected head with a single hidden layer as a classifier. Don't know what these things mean? Not to worry, we will dive deeper in the coming lessons. For the moment you need to know that we are building a model which will take images as input and will output the predicted probability for each of the categories (in this case, it will have 37 ouptuts).

We will train for 10 epochs (10 cycles through all our data).

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.save('stage-1_sz-150')

### Visualize and Analyze

Let's see what results we have got. 

We will first see which were the categories that the model most confused with one another. We will try to see if what the model predicted was reasonable or not. In this case the mistakes look reasonable (none of the mistakes seems obviously naive). This is an indicator that our classifier is working correctly. 

Furthermore, when we plot the confusion matrix, we can see that the distribution is heavily skewed: the model makes the same mistakes over and over again but it rarely confuses other categories. This suggests that it just finds it difficult to distinguish some specific categories between each other; this is normal behaviour.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=80)

In [ ]:
# This is for accuracy in top 3 -
def accuracy_topk(output, target, topk=(3,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
# FOR TOP-3 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(3,))

In [ ]:
# FOR TOP-5 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(5,))

## Unfreezing, fine-tuning, and learning rates

Since our model is working as we expect it to, we will *unfreeze* our model and train some more.

In [ ]:
learn.load('stage-1_sz-150')
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20, max_lr=slice(3e-7,3e-5))

In [ ]:
learn.save('stage-1-34_sz-150')

### Visualize and Analyze

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=80)

In [ ]:
# FOR TOP-3 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(3,))

In [ ]:
# FOR TOP-5 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(5,))

## Training: resnet50

Now we will train in the same way as before but with one caveat: instead of using resnet34 as our backbone we will use resnet50 (resnet34 is a 34 layer residual network while resnet50 has 50 layers. Later in the course you can learn the details in the [resnet paper](https://arxiv.org/pdf/1512.03385.pdf)).

Basically, resnet50 usually performs better because it is a deeper network with more parameters. Let's see if we can achieve a higher performance here.

In [ ]:
data = ImageDataBunch.from_csv(path, csv_labels="train_labels.csv" , ds_tfms=get_transforms(), size=150)
data.normalize(imagenet_stats)

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy)

In [ ]:
#Before Unfreezing let's see how much effetive is resnet50
learn.fit_one_cycle(10)

In [ ]:
# let's save model for further unfreeze training
learn.save('stage-1_arch-50_sz-150')

### Visualize and Analyze

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=80)

In [ ]:
# This is for accuracy in top 3 -
def accuracy_topk(output, target, topk=(3,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
# FOR TOP-3 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(3,))

In [ ]:
# FOR TOP-5 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(5,))

## Unfreezing, fine-tuning, and learning rates

Since our model is working as we expect it to, we will *unfreeze* our model and train some more.

In [ ]:
learn.load('stage-1_arch-50_sz-150')
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:

learn.fit_one_cycle(10, max_lr=slice(1e-6,1e-5))

In [ ]:
learn.save('stage-1_arch-50_sz-150-unfreeze')

### Visualize and Analyze

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=80)

In [ ]:
# FOR TOP-3 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(3,))

In [ ]:
# FOR TOP-5 Accuracy
accuracy_topk(output=interp.y_pred, target=interp.y_true, topk=(5,))

## Training: resnet152

Now we will train in the same way as before but with one caveat: instead of using resnet34 as our backbone we will use resnet152 (resnet34 is a 34 layer residual network while resnet152 has 152 layers. Later in the course you can learn the details in the [resnet paper](https://arxiv.org/pdf/1512.03385.pdf)).

Basically, resnet152 usually performs better because it is a deeper network with more parameters. Let's see if we can achieve a higher performance here.

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=get_transforms(), size=299, bs=32)
data.normalize(imagenet_stats)

In [ ]:
learn = create_cnn(data, models.resnet152, metrics=accuracy)

In [ ]:
#Before Unfreezing let's see how much effetive is resnet100
learn.fit_one_cycle(5)

In [ ]:
# let's save model for further unfreez training
learn.save('stage-1-100')

In [ ]:
# let's find optimal learning rate
learn.lr_find()
learn.recorder.plot()

In [ ]:
# Unfreeze Resnet to apply new optimal learning rate
learn.unfreeze()
# train for 1 cycle to see if metrics is improving
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

In [ ]:
# It did improved so let's train it for 5 more cycle
learn.fit_one_cycle(5, max_lr=slice(1e-6,1e-4))